# Model Playground

This notebook is intended as a playground to test out different models with the data prepared from the previous notebook.

## Import Modules

In [15]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras import layers
import tensorflow as tf
import datetime
import cv2

## CNN Model Training
This code trains a convolutional neural network to perform obstacle detection. Here is a breakdown of the implementation:

- The model architecture is defined using the Sequential API of Keras. The model consists of a series of convolutional layers with ReLU activation and pooling layers, followed by two fully connected layers. The architecture is chosen such that the spatial dimensions of the feature maps are progressively reduced by the pooling layers.
- 5 pooling layers with a pool size of 2x2 are used to progressively reduce the spatial dimensions of the feature maps.
- The binary_crossentropy loss function is commonly used in binary classification tasks as it is well suited for computing the cross-entropy between two probability distributions.
- The Adam optimizer is commonly used in deep learning as it adapts its learning rate during training, making it more efficient and effective compared to other optimization algorithms.
- In obstacle detection, it's important to have a metric that evaluates the model's ability to detect obstacles accurately. F1 score is a good metric to use because it takes both precision and recall into account, which makes it more reliable when the class distribution is imbalanced.

In [14]:
# Define model
model_name = "obstacle_detection_v3"

# Define checkpoint callback
checkpoint_dir = f"checkpoint/{model_name}"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
filepath=os.path.join("./checkpoint/", "ckpt_{epoch}"),
save_weights_only=True,
save_freq="epoch")

#Define logger callback
log_dir = f"logs/{model_name}/{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#Load Data
X = np.load("data/train_x.npy", allow_pickle=True)
y = np.load("data/train_y_v2.npy", allow_pickle=True) #Labels from Algorithm 2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

#Define model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=X_train.shape[1:]),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    #representing the binary classification output (obstacle or no obstacle)
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

csv_logger = tf.keras.callbacks.CSVLogger('training.log', separator=',', append=False)

# Train model with CSVLogger callback
model.fit(X_train, y_train, epochs=10, batch_size=32,
          callbacks=[checkpoint_callback, tensorboard_callback, csv_logger],
          validation_data=(X_test, y_test))

# Evaluate the model on the testing set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test Accuracy:', test_acc)

#Save the model
model.save(f"{model_name}.h5")
weights = model.get_weights()
np.savetxt(f"{model_name}_weights.txt", np.concatenate([w.flatten() for w in weights]))
model.summary()

Epoch 1/10
17/17 [==============================] - 41s 2s/step - loss: 29.5220 - accuracy: 0.7008 - val_loss: 0.5572 - val_accuracy: 0.8161
Epoch 2/10
17/17 [==============================] - 41s 2s/step - loss: 0.4680 - accuracy: 0.8185 - val_loss: 0.4184 - val_accuracy: 0.8161
Epoch 3/10
17/17 [==============================] - 40s 2s/step - loss: 0.4131 - accuracy: 0.8263 - val_loss: 0.3885 - val_accuracy: 0.8296
Epoch 4/10
17/17 [==============================] - 49s 3s/step - loss: 0.3894 - accuracy: 0.8320 - val_loss: 0.3800 - val_accuracy: 0.8251
Epoch 5/10
17/17 [==============================] - 42s 2s/step - loss: 0.3815 - accuracy: 0.8340 - val_loss: 0.3894 - val_accuracy: 0.8430
Epoch 6/10
17/17 [==============================] - 42s 2s/step - loss: 0.3612 - accuracy: 0.8668 - val_loss: 0.4245 - val_accuracy: 0.8341
Epoch 7/10
17/17 [==============================] - 42s 2s/step - loss: 0.3947 - accuracy: 0.8533 - val_loss: 0.3640 - val_accuracy: 0.8386
Epoch 8/10
17/17 [=

In [16]:
from sklearn.metrics import f1_score

model = load_model('obstacle_detection_v3.h5')

# Define the input directory and true labels
input_dir = 'data/validate'
true_labels = np.load('data/test_y_v2.npy')

# Get a list of all image files in the directory and sort them alphabetically
image_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.png')])

# Initialize empty arrays to store predicted and true labels
predicted_labels = np.zeros(len(image_files))

# Loop over each image file and make predictions
for i, filename in enumerate(image_files):
    # Load the image and preprocess it as needed
    image_path = os.path.join(input_dir, filename)
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    data = np.array(image)
    data = np.expand_dims(data, axis=0)  # shape: (1, 384, 288, 4)

    # Make predictions on the image
    predictions = model.predict(data)

    # Print the predicted classes
    predicted_class = np.argmax(predictions, axis=1)[0]
    print(f"File {filename}: predicted class {predicted_class}")
    
    # Store predicted label
    predicted_labels[i] = predicted_class

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels)

print(f"True labels: {true_labels}")
print(f"Predicted labels: {predicted_labels}")
print(f"F1 score: {f1}")

1/1 [==============================] - 0s 359ms/step
File ground_truth.png: predicted class 0
1/1 [==============================] - 0s 43ms/step
File no_obstacle.png: predicted class 0
1/1 [==============================] - 0s 39ms/step
File side_obstacle1.png: predicted class 0
1/1 [==============================] - 0s 27ms/step
File side_obstacle2.png: predicted class 0
True labels: [1 0 1 1]
Predicted labels: [0. 0. 0. 0.]
F1 score: 0.0
